# Model testing and approbation

In [2]:
#Loading dependencies
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
%matplotlib inline

## Loading the checkpoint

A function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
def load_model(path, orig_mod, classifier='fc', device='cpu'): 

    loaded = torch.load(path, map_location=device)
    #Define the name of fully-connected layer
    if classifier == 'fc':
  
        orig_mod.fc = loaded['classifier']
  
    else: 
    
        orig_mod.classifier = loaded['classifier']
    #Freezing model parameters
    for param in orig_mod.parameters():
        param.requires_grad = False

    orig_mod.load_state_dict(loaded['state_dict'])
    return orig_mod

upload_model = load_model('model_flowers.pt', models.resnet50(pretrained=True), 'fc', 'cpu')

## Image Preprocessing

It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, the function resizes the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be  Then, it crops out the center 224x224 portion of the image.

Color channels of images are converted to the expected parameters of the pre-trained model -  floats 0-1. need to convert the values.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    Process a PIL image for use in a PyTorch model
    im = Image.open(image)
    resized = im.resize((400,256)).crop((88,16,312,240))
    np_image = ((np.array(resized)/255)-np.array([0.485, 0.456, 0.406]))/np.array([0.229, 0.224, 0.225])
    image = torch.from_numpy(np.transpose(np_image, (2,1,0)))
    return image
  
tensor = process_image('flower_data/train/1/image_06773.jpg')
#print(type(tensor))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax
  
imshow(tensor)

## Class Prediction

The function that returns k-top prediction values. It takes a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    image = process_image(image_path).unsqueeze_(0)
    model.eval()
    model.class_to_idx = train_data.class_to_idx
    model.to('cuda')
    output = model(image.to(device='cuda', dtype=torch.float))
    
    # Top probs
    top_probs, top_labs = output.topk(topk)
    top_probs = top_probs.detach().cpu().numpy().tolist()[0] 
    top_labs = top_labs.detach().cpu().numpy().tolist()[0]
    
    # Convert indices to classes
    idx_to_class = {val: key for key, val in    
                                      model.class_to_idx.items()}
    top_labels = [idx_to_class[lab] for lab in top_labs]
    top_flowers = [cat_to_name[idx_to_class[lab]] for lab in top_labs]
    
    return top_probs, top_labels, top_flowers

prediction = predict('flower_data/train/1/image_06773.jpg', upload_model, 5)

prediction

## Sanity Checking

Function that graphs top-n probabilities distribution, along with visualization of image and its label name.

<img src='assets/inference_example.png' width=300px>


In [ ]:
def plot_solution(image_path, model):
    # Set up plot
    plt.figure(figsize = (6,10))
    ax = plt.subplot(2,1,1)
    # Set up title
    flower_num = image_path.split('/')[2]
    title_ = cat_to_name[flower_num]
    # Plot flower
    img = process_image(image_path)
    imshow(img, ax, title = title_);
    # Make prediction
    probs, labs, flowers = predict(image_path, model) 
    # Plot bar chart
    plt.subplot(2,1,2)
    sns.barplot(x=probs, y=flowers, color=sns.color_palette()[0]);
    plt.show()
    
plot_solution('flower_data/train/1/image_06773.jpg', upload_model)